In [ ]:
!pip install transformers
!pip install torch

In [ ]:
import torch
import os

# Set the API key
os.environ['ACCESS_KEY'] = ''

In [ ]:
api_key = os.getenv('ACCESS_KEY')
tokenizer = AutoTokenizer.from_pretrained("mistralai/mathstral-7B-v0.1",use_auth_token=api_key)
model = AutoModelForCausalLM.from_pretrained("mistralai/mathstral-7B-v0.1",use_auth_token=api_key)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/588k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
def prepare_prompt(question, options, custom_prompt):
    options_str = " ".join([f"{chr(65+i)}) {option}" for i, option in enumerate(options)])
    prompt = f"{custom_prompt} **Question:** {question} **Options:** {options_str} **Important:** The last line of your answer should contain only the single letter corresponding to the best option, with no additional text."
    return prompt


In [ ]:
def tokenize_input(prompt, tokenizer):
    return tokenizer.encode(prompt, return_tensors="pt")

In [ ]:
def generate_response(input_ids, model, tokenizer):
    response = model.generate(input_ids, max_length=2000)
    generated_text = tokenizer.decode(response[0], skip_special_tokens=True)
    return generated_text


In [ ]:
def extract_answer(generated_text):
    match = re.search(r"([A-D])\s*$", generated_text)
    if match:
        return match.group(1)
    return "No valid answer found"


In [ ]:
# Evaluate the answer
def evaluate_answer(predicted_answer, gt_answer):
    return "Correct!" if predicted_answer == gt_answer else "Incorrect."



In [ ]:
  import pandas as pd

# Load the dataset from the TSV file
df = pd.read_csv('/content/cti-mcq.tsv', delimiter='\t')

# Prepare lists to store results
results = []

# Process only the first 10 rows in the dataset
for index, row in df.iterrows():
    question = row['Question']
    options = [row['Option A'], row['Option B'], row['Option C'], row['Option D']]
    prompt = row['Prompt']
    gt = row['GT']

    # Prepare the prompt
    formatted_prompt = prepare_prompt(question, options, prompt)

    # Tokenize the input
    input_ids = tokenize_input(formatted_prompt, tokenizer)

    # Generate a response
    generated_text = generate_response(input_ids, model, tokenizer)

    # Extract the answer
    predicted_answer = extract_answer(generated_text)

    # Evaluate the answer
    evaluation = evaluate_answer(predicted_answer, gt)

    # Append results
    results.append({
        'Question': question,
        'Predicted Answer': predicted_answer,
        'Ground Truth': gt,
        'Evaluation': evaluation
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv('predicted_answers.csv', index=False)

print("Predicted answers saved to 'predicted_answers.csv'")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio